In [145]:
import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time
import pickle,h5py,json

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D,Conv2DTranspose,UpSampling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap,TSNE
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from scipy import fftpack
from scipy.signal import resample,correlate
from xgboost import XGBClassifier,XGBRegressor

import gym

import pandas_datareader as pdr
import networkx as nx
#from lmfit import Model

import matplotlib.image as mpimg
from scipy import ndimage 

warnings.filterwarnings('ignore')
sns.set()

In [105]:
def blur_img(img,blur_factor):
    blurred_img = img.copy()
    for cdim in range(img.shape[2]):
        blurred_img[:,:,cdim] = ndimage.uniform_filter(img[:,:,cdim],size=blur_factor)
    return blurred_img

In [106]:
def rand_blur(img,MAXBLUR=5,ADDEDBLUR=4):
    blur0 = rnd.randint(0,MAXBLUR)
    blur1 = blur0 + ADDEDBLUR
    img0 = blur_img(img,blur_factor=blur0)
    img1 = blur_img(img,blur_factor=blur1)
    return img0,img1

In [107]:
def build_training_set(base_path='..\\..\\TF_data\\SAT\\train-jpg',limited_load=-1):
    available_img = os.listdir(base_path)
    if limited_load>0: available_img = available_img[:limited_load]
    X = []
    Y = []
    for terminal_path in available_img:
        img = mpimg.imread(base_path+'\\'+terminal_path)
        img0,img1 = rand_blur(img)
        Y.append(img0)
        X.append(img1)
    X = np.stack(X,axis=0)
    Y = np.stack(Y,axis=0)
    return X,Y

In [108]:
X,Y = build_training_set(limited_load=1000)

In [168]:
img_heigth = X.shape[1]
img_width = X.shape[2]
img_channels = X.shape[3]

in0 = Input(shape=(img_heigth,img_width,img_channels,))
x0 = Conv2D(filters=32,kernel_size=(16,16),activation='relu')(in0)
x0 = MaxPooling2D(pool_size=(2,2),strides=(1,1))(x0)
x0 = Conv2D(filters=16,kernel_size=(8,8),activation='relu')(x0)
x0 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(x0)
x0 = Dropout(rate=0.15)(x0)
x0 = Conv2D(filters=16,kernel_size=(16,16),activation='relu')(x0)
x0 = MaxPooling2D(pool_size=(2,2),strides=(1,1))(x0)
x0 = Conv2D(filters=8,kernel_size=(16,16),activation='relu')(x0)
x0 = MaxPooling2D(pool_size=(2,2),strides=(2,2))(x0)
x0 = Dropout(rate=0.15)(x0)
x0 = Conv2D(filters=4,kernel_size=(8,8),activation='relu')(x0)
enc0 = MaxPooling2D(pool_size=(2,2),strides=(1,1))(x0)

x0 = UpSampling2D(size=(2,2))(enc0)
x0 = Conv2DTranspose(filters=32,kernel_size=(8,8),activation='relu')(x0)
x0 = Dropout(rate=0.15)(x0)
x0 = Conv2DTranspose(filters=64,kernel_size=(16,16),activation='relu')(x0)
x0 = Conv2DTranspose(filters=128,kernel_size=(16,16),activation='relu')(x0)
x0 = UpSampling2D(size=(2,2))(x0)
x0 = Dropout(rate=0.15)(x0)
x0 = Conv2DTranspose(filters=32,kernel_size=(32,32),activation='relu')(x0)
x0 = Conv2DTranspose(filters=4,kernel_size=(16,16),activation='relu')(x0)
dec0 = Dropout(rate=0.15)(x0)

enc_model = Model(input=in0,output=enc0)
dec_model = Model(input=in0,output=dec0)

dec_model.compile(optimizer='adam',loss='mse',metrics=['mse'])

In [169]:
dec_model.fit(X,Y,batch_size=3,epochs=3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 256, 256, 4)       0         
_________________________________________________________________
conv2d_107 (Conv2D)          (None, 241, 241, 32)      32800     
_________________________________________________________________
max_pooling2d_107 (MaxPoolin (None, 240, 240, 32)      0         
_________________________________________________________________
conv2d_108 (Conv2D)          (None, 233, 233, 16)      32784     
_________________________________________________________________
max_pooling2d_108 (MaxPoolin (None, 116, 116, 16)      0         
_________________________________________________________________
dropout_60 (Dropout)         (None, 116, 116, 16)      0         
_________________________________________________________________
conv2d_109 (Conv2D)          (None, 101, 101, 16)      65552     
__________